## Mounting

In [14]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/PRML/Assignment_1

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/PRML/Assignment_1


## Import packages + declare paths

In [2]:
%matplotlib inline
!pip install ipython-autotime
%load_ext autotime

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.utils import shuffle
import os
from PIL import Image
import time

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, MaxPool2D, BatchNormalization
from tensorflow.keras import optimizers
from keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.losses import CategoricalCrossentropy
IMG_TRAIN_PATH = '/content/drive/My Drive/Colab Notebooks/PRML/Assignment_1/train'
DATA_NP_PATH = '/content/drive/My Drive/Colab Notebooks/PRML/Assignment_1/np_data'


time: 1.99 s


## Function to load training data

In [3]:
def load_data_numpy(dir):
    data_dir = os.path.join(dir, "data_train.npy")
    label_dir = os.path.join(dir, "label_train.npy")
    return np.load(data_dir), np.load(label_dir)


def load_data_image(dir):
    LABEL_LIST = [10, 1, 2, 3, 4, 5, 6, 7, 8, 9]

    X = []
    y_onehot = np.zeros((60000, 10))

    for num in range(10):
        sub_dir = os.path.join(dir, str(LABEL_LIST[num]))
        for img_dir in os.listdir(sub_dir):
            img = Image.open(os.path.join(sub_dir, img_dir))
            array_img = np.array(img)
            X.append(np.array(img))

        y_onehot[6000*(num):6000*(num+1), num] = 1

        # Show example of number
        # img.show(title=str(num+1))
        plt.figure()
        plt.title(str(LABEL_LIST[num]))
        plt.imshow(array_img)
        plt.show()

    X = np.reshape(np.array(X), (60000, 28, 28, 1))
    X_shuffle, y_shuffle = shuffle(X, y_onehot, random_state=1)

    np.save('data_train.npy', X_shuffle)
    np.save('label_train.npy', y_shuffle)

    return X_shuffle, y_shuffle

time: 24.8 ms


In [5]:
X,y = load_data_numpy(DATA_NP_PATH)
print(X.shape)
print(y.shape)

(60000, 28, 28, 1)
(60000, 10)
time: 4 s


In [6]:
X_train = X[:50000, :, :]
y_train = y[:50000, :]

X_small = X_train[:1000, :, :]
y_small = y[:1000, :]

X_test = X[50000:, :, :]
y_test = y[50000:, :]

time: 1.86 ms


# Load evaluation data

In [7]:
def load_eval_data_numpy(dir):
    data_dir = os.path.join(dir, "eval_data.npy") 
    return np.load(data_dir)

time: 1.3 ms


In [8]:
X_eval = load_eval_data_numpy(DATA_NP_PATH)
print(X_eval.shape)

(10000, 28, 28, 1)
time: 2 s


# CNN

In [35]:
def build_CNN(reg):
    model_cnn = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        BatchNormalization(),
        #Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),
        BatchNormalization(),
        #Dropout(0.25),
        
        Flatten(),
        Dense(512, activation='relu'),
        #Dropout(0.5),
        Dense(10, activation='softmax', activity_regularizer=l2(reg))
    ])
    #Adam, batch_size, #nesterov=True, momentum=0.9

    opt_cnn = optimizers.Adam(learning_rate=1e-4)
    # good learning_rate = 1e-4
    model_cnn.compile(optimizer=opt_cnn, loss=CategoricalCrossentropy(), metrics=['accuracy'])
    model_cnn.summary()

    return model_cnn

time: 17.2 ms


In [42]:
model_cnn = build_CNN(1e-4)
model_cnn.fit(X, y, validation_split=0.2, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 13, 13, 32)        128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0

time: 19min 25s


## Check on training set X,y

In [44]:
test_loss, test_acc = model_cnn.evaluate(X, y)
print("Loss:", test_loss, "Accuracy:", test_acc)

1875/1875 [==============================] - 4s 2ms/step - loss: 0.0045 - accuracy: 0.9993
Loss: 0.004469430074095726 Accuracy: 0.9993166923522949
time: 4.25 s


## Tunning parameters

In [45]:
reg_list = {}
for i in range(15): 
    if i<5:
        reg = np.random.uniform(5e-1, 1e-1)
    elif i<10:
        reg = np.random.uniform(1e-1, 5e-2)
    else:
        reg = np.random.uniform(5e-2, 1e-2)
    
    model_cnn = build_CNN(reg)
    history = model_cnn.fit(X, y, validation_split=0.2, epochs=200, verbose=0)  # epochs=150-200 is good
    reg_list[reg] = [history.history['loss'][-1], history.history['val_accuracy'][-1]]
    print(i, reg, reg_list[reg])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)         

## Applied a suitable regularization value

In [ ]:
model_cnn = build_CNN(1e-4)
model_cnn.fit(X, y, validation_split=0.2, epochs=200, verbose=0)
import h5py
model_cnn.save("cnn_model_9_11.h5")
from tensorflow.keras.models import load_model
new_model = load_model("/content/drive/My Drive/Colab Notebooks/PRML/Assignment_1/cnn_model.h5")
new_model.summary()

In [10]:
test_loss, test_acc = new_model.evaluate(X, y)
print("Loss:", test_loss, "Accuracy:", test_acc)

1875/1875 [==============================] - 5s 2ms/step - loss: 0.0241 - accuracy: 0.9991
Loss: 0.024081509560346603 Accuracy: 0.9991000294685364
time: 11.4 s


In [11]:
y_prob = new_model.predict(X_eval)
print(y_prob.shape)
y_eval = np.argmax(y_prob,axis=1)
print(y_eval.shape)
print(y_eval[:10])

(10000, 10)
(10000,)
[9 1 1 4 7 3 2 3 5 7]
time: 512 ms


## Change labels 0 to 10

In [24]:
y_eval[y_eval == 0] = 10
list_dir = os.listdir(os.path.join(os.getcwd(),'evaluate/'))
print(len(list_dir))
print(list_dir[:20]) # listdir does not list directories in correct order !

[ 1  2  3  4  5  6  7  8  9 10]
[ 9  1  1  4  7  3  2  3  5  7 10  3  7  7  7  9  4 10  4  6]
[ 1  2  3  4  5  6  7  8  9 10]
[ 9  1  1  4  7  3  2  3  5  7 10  3  7  7  7  9  4 10  4  6]
10000
['09951.jpg', '09557.jpg', '09208.jpg', '09325.jpg', '09394.jpg', '09531.jpg', '09578.jpg', '09185.jpg', '09611.jpg', '09903.jpg', '09503.jpg', '09804.jpg', '09344.jpg', '09184.jpg', '09299.jpg', '09599.jpg', '09916.jpg', '09132.jpg', '09957.jpg', '09832.jpg']
time: 55.8 ms


## Save result

In [50]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/PRML/Assignment_1/
img_id = np.arange(0,10000,1)
print(y_eval[:20])
result = np.column_stack([img_id,y_eval])
print(result[:20])
print('unique values: ',np.unique(result[:,1]))
import pandas as pd
result = pd.DataFrame(data=result,columns=['Id','Category']).astype(int)
result.to_csv('result_pd.csv',index=False,sep=';')


/content/drive/My Drive/Colab Notebooks/PRML/Assignment_1
[ 9  1  1  4  7  3  2  3  5  7 10  3  7  7  7  9  4 10  4  6]
[[ 0  9]
 [ 1  1]
 [ 2  1]
 [ 3  4]
 [ 4  7]
 [ 5  3]
 [ 6  2]
 [ 7  3]
 [ 8  5]
 [ 9  7]
 [10 10]
 [11  3]
 [12  7]
 [13  7]
 [14  7]
 [15  9]
 [16  4]
 [17 10]
 [18  4]
 [19  6]]
unique values:  [ 1  2  3  4  5  6  7  8  9 10]
time: 31.5 ms
